# Movie recommendation project

In [15]:
import pandas as pd
import warnings
import tkinter as tk
import warnings
warnings.filterwarnings('ignore')

In [ ]:
cols='user_id item_id rating timestamp'.split()
df=pd.read_csv('u.data',sep='\t',names=cols)

cols_movies=['item_id','title']+[str(i) for i in range(22)]
movie_titles = pd.read_csv('u.item',sep='|',names=cols_movies)
movie_titles.head()
movie_titles = movie_titles[['item_id','title']]
data=pd.merge(df,movie_titles,on='item_id')
rating = pd.DataFrame(data.groupby('title')['rating'].mean())
rating['count'] = data['title'].value_counts()
pivot_df =data.pivot_table(index='user_id',columns='title', values='rating')



app=tk.Tk()
app.title('Recommendation System')
app.geometry('350x200')

#Creating Labels
tk.Label(app,text='Movie you Watched:',bg='#ffffb3',font=('Roboto Slab',10)).place(x=25,y=25)
tk.Label(app,text='Recommended: ',bg='#ffe6ee',font=('Roboto Slab', 10, 'bold')).place(x=25,y=100)
tk.Label(app,text='You can also try:',bg='#ffe6ee',font=('Roboto Slab', 10, 'bold')).place(x=25,y=125)

movie_var = tk.Variable(app)
tk.Entry(app,textvariable=movie_var,width=25).place(x=150,y=25)

recommend_var = tk.Variable(app)
recommend_var.set('    Enter movie name..')
tk.Label(app,textvariable=recommend_var).place(x=150,y=100)

you_can_also_try_var = tk.Variable(app)
you_can_also_try_var.set('    Enter movie name..')
tk.Label(app,textvariable=you_can_also_try_var).place(x=150,y=125)

def find_recommendation():
    movie=movie_var.get().lower().strip()
    try:
        movie=movie_titles['title'][movie_titles['title'].apply(lambda x: movie in x.lower())].values[0]
        movie_var.set(movie)
    except IndexError:
        recommend_var.set('    Movie not found')
        you_can_also_try_var.set('    Movie not found')
    else:
        print('Now we will find the suggestions')
        corr_df = pd.DataFrame(pivot_df.corrwith(pivot_df[movie]),columns=['Correlation'])
        corr_df.dropna(inplace=True)
        corr_df =corr_df.join(rating['count'])
        recommend_var.set(corr_df[corr_df['count']>200].sort_values(by='Correlation',ascending=False).index[1])
        you_can_also_try_var.set(corr_df[corr_df['Correlation']>0.4].sort_values(by='Correlation',ascending=False).index[0])
        print(recommend_movie)
        print(you_can_also_try)
        
tk.Button(app,text='Find Suggestion',command=find_recommendation,bg='#1a1aff',fg='#ffffff').place(x=125,y=60)

app.mainloop()